In [1]:
from fipy import *

In [2]:
steps = 100
N = 60
L = 2.5 * N / 100.
dL = L / N
mesh = Grid2D(dx = dL, dy = dL, nx = N, ny = N)


In [3]:
timeStepDuration = 0.02
phaseTransientCoeff = 0.1
thetaSmallValue = 1e-6
beta = 1e5
mu = 1e3
thetaTransientCoeff = 0.01
gamma = 1e3
epsilon = 0.008
s = 0.01
phi0 = 0.5
alpha = 0.015
#alpha = 1.
D = a = 0.015

The system is held isothermal at

In [4]:
temperature = 0

and is initialized to liquid everywhere:

In [5]:
phase = CellVariable(name = 'phase field', mesh = mesh)
psi = CellVariable(name = r'$\psi$', mesh = mesh)

The orientation is initialized to a uniform value to denote the randomly 
oriented liquid phase.

In [6]:
theta = ModularVariable(name=r'$\theta$',mesh=mesh, value=-numerix.pi+0.0001,hasOld=True)

Define different solid circular domains by random seeds

In [7]:
x, y = mesh.cellCenters
numSeeds = 5
numerix.random.seed(4235)
for a, b, thetaValue in numerix.random.random([numSeeds,3]):
#m = [[L/2.,L/2.,0.2],[L/2.,L/2.,0.3]]
#for a, b, thetaValue in m:
    radius = dL * 2.
    segment = ((x-a*N*dL)**2 + (y-b*N*dL)**2) < radius
    phase[segment] = 1.
    theta[segment] = numerix.pi * (2*thetaValue-1)
    

Here we are solving Cahn-Hilliard equation comming from KWC model:
 
$$\tau_{\phi}\frac{\partial\phi}{\partial\theta}=\nabla\left (D\nabla \frac{\delta F}{\delta \phi} \right)=D \nabla^2\left(-\alpha^2 \nabla^2 \phi+\frac{\partial f}{\partial \phi}+\frac{\partial g}{\partial\phi}s|\nabla \theta| + \frac{\partial h}{\partial \phi}\frac{\epsilon^2}{2}|\nabla \theta|^2 \right)$$

where:

$$f(\phi) = a*\phi^2(1-\phi)^2$$



In [8]:
def buildPhaseEquation(phase, theta):
    mPhiVar1 = a**2 * phase *(1 - phase) * (1 - 2 * phase)
    mPhiVar2 = a**2 * (1 - 6 * phase*(1-phase))
    #mPhiVar1 = (phase - 0.5 + temperature) * phase *(1-phase)
    #mPhiVar2 =  phase*(1-phase)+(phase - 0.5 + temperature)*(1-2*phase)
    thetaMag = theta.old.grad.mag
    implicitSource = mPhiVar2
    implicitSource -= (2*s + epsilon**2 * thetaMag)*thetaMag
    
    eq1 = TransientTerm(var=phase,coeff=phaseTransientCoeff) == \
            DiffusionTerm(coeff=D,var=psi) 
    eq2 = ImplicitSourceTerm(coeff=1., var=psi) == \
            -DiffusionTerm(coeff=alpha**2,var=phase) \
            +ImplicitSourceTerm(coeff=implicitSource, var=phase) \
            +mPhiVar1 - mPhiVar2 * phase
    return eq1 & eq2

In [9]:
phaseEq = buildPhaseEquation(phase, theta)

The $\theta$ equation is built in the following way. The details for this equation
are provided by J.A.Warren *et.al. The main detail is that a source must be added
to correct for discretization of $theta$ on the circle. The source term requires 
the evaluation of the face gradient without modular operator.

In [10]:
def buildThetaEquation(phase, theta):
    phaseMod = phase + ( phase < thetaSmallValue ) * thetaSmallValue
    phaseModSq = phaseMod * phaseMod
    expo = epsilon * beta * theta.grad.mag
    expo = (expo < 100.) * (expo - 100.)+100.
    pFunc = 1. + numerix.exp(-expo) * (mu / epsilon - 1.)
    
    phaseFace = phase.arithmeticFaceValue
    phaseFaceSq = phaseFace * phaseFace
    gradMag = theta.faceGrad.mag
    eps = 1. / gamma / 10.
    gradMag += (gradMag < eps) * eps
    IGamma = (gradMag > 1. / gamma) * ( 1 * gradMag - gamma ) + gamma
    diffusionCoeff = phaseFaceSq * (s * IGamma + epsilon**2)
    
    thetaGradDiff = theta.faceGrad - theta.faceGradNoMod
    sourceCoeff = (diffusionCoeff * thetaGradDiff).divergence
    
    return TransientTerm(thetaTransientCoeff * phaseModSq * pFunc) == \
        DiffusionTerm(diffusionCoeff) + sourceCoeff
    

In [11]:
thetaEq = buildThetaEquation(phase, theta)

If example is run interactively, we create viewers for the phase and orientation variables.
Rather than viewing the raw orientation, which is not meaningful in the liquid phase, we weight
the orientation by the phase.

In [12]:
if __name__ == '__main__':
    phaseViewer = Viewer(vars=phase,datamin=0,datamax=1.)
    thetaProd = -numerix.pi + phase * (theta + numerix.pi)
    thetaProductViewer = Viewer(vars=theta, datamin=-numerix.pi, datamax=numerix.pi)
    phaseViewer.plot()
    thetaProductViewer.plot()
    

We step the solution in time, plotting as we go interactively

In [13]:
for i in range(steps):
    theta.updateOld()
    thetaEq.solve(theta, dt=timeStepDuration)
    phaseEq.solve(dt=timeStepDuration)
    if __name__ == '__main__':
        phaseViewer.plot()
        thetaProductViewer.plot()
        print "step: ", i
        

step:  0
step:  1
step:  2
step:  3
step:  4
step:  5
step:  6
step:  7
step:  8
step:  9
step:  10
step:  11
step:  12
step:  13
step:  14
step:  15
step:  16
step:  17
step:  18
step:  19
step:  20
step:  21
step:  22
step:  23
step:  24
step:  25
step:  26
step:  27
step:  28
step:  29
step:  30
step:  31
step:  32
step:  33
step:  34
step:  35
step:  36
step:  37
step:  38
step:  39
step:  40
step:  41
step:  42
step:  43
step:  44
step:  45
step:  46
step:  47
step:  48
step:  49
step:  50
step:  51
step:  52
step:  53
step:  54
step:  55
step:  56
step:  57
step:  58
step:  59
step:  60
step:  61
step:  62
step:  63
step:  64
step:  65
step:  66
step:  67
step:  68
step:  69
step:  70
step:  71
step:  72
step:  73
step:  74
step:  75
step:  76
step:  77
step:  78
step:  79
step:  80
step:  81
step:  82
step:  83
step:  84
step:  85
step:  86
step:  87
step:  88
step:  89
step:  90
step:  91
step:  92
step:  93
step:  94
step:  95
step:  96
step:  97
step:  98
step:  99
